In [21]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from sklearn import metrics
import gc

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/amex-default-prediction/sample_submission.csv
/kaggle/input/amex-default-prediction/train_data.csv
/kaggle/input/amex-default-prediction/test_data.csv
/kaggle/input/amex-default-prediction/train_labels.csv
/kaggle/input/amex-parquet/test_data.parquet
/kaggle/input/amex-parquet/train_data.parquet


In [22]:
df1=pd.read_parquet('/kaggle/input/amex-parquet/train_data.parquet')
df1.head()
df1.shape
df_label = pd.read_csv('/kaggle/input/amex-default-prediction/train_labels.csv')
df_label.head()

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3552, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_17/3445842997.py", line 1, in <module>
    df1=pd.read_parquet('/kaggle/input/amex-parquet/train_data.parquet')
  File "/opt/conda/lib/python3.7/site-packages/pandas/io/parquet.py", line 500, in read_parquet
    **kwargs,
  File "/opt/conda/lib/python3.7/site-packages/pandas/io/parquet.py", line 240, in read
    path_or_handle, columns=columns, **kwargs
  File "/opt/conda/lib/python3.7/site-packages/pyarrow/parquet/__init__.py", line 2781, in read_table
    use_pandas_metadata=use_pandas_metadata)
  File "/opt/conda/lib/python3.7/site-packages/pyarrow/parquet/__init__.py", line 2445, in read
    use_threads=use_threads
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/

TypeError: object of type 'NoneType' has no len()

In [ ]:
df1.head()

In [ ]:
#checking out data types
print(df1.dtypes.value_counts())
list(df1.select_dtypes(['object']).columns)

In [ ]:
#visualizing columns

#hist = df1.hist(bins=10, figsize = (40,200), layout=(-1,4) )

In [ ]:
print(df1.shape[1])
#inspecting NaN
for i in range(len(df1.columns)):
    if (df1.iloc[:,i].isnull().sum()/len(df1) > 0.1):
        print(df1.columns[i], round(df1.iloc[:,i].isnull().sum()/len(df1),2))

#drop columns with high freq of NaN
columns_to_drop = [column for column in df1.columns if df1[column].isnull().sum()/len(df1) >= 0.1]
df1.drop(columns_to_drop, axis=1, inplace=True)
print(df1.shape[1])

In [ ]:
#using only most recent transaction from each customer
temp = df1.shape
df1=df1.set_index(['customer_ID'])
df1=df1.ffill()
df1=df1.bfill()
df1=df1.reset_index()

df1=df1.groupby('customer_ID').tail(1)
df1=df1.set_index(['customer_ID'])

#Drop date column since it is no longer useful
df1.drop(['S_2'],axis=1,inplace=True)

print(temp, df1.shape)

#inspecting NaN
for i in range(len(df1.columns)):
    print('Columns left with NaN:')
    if (df1.iloc[:,i].isnull().sum()/len(df1) > 0):
        print(df1.columns[i], round(df1.iloc[:,i].isnull().sum()/len(df1),2))

In [ ]:
df1.head()
df1 = df1.drop(['target'], axis=1)
keep = df1.columns

In [ ]:
print(df1.shape)
# Create correlation matrix
corr_matrix = df1.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.9
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]

# Drop features w/ high correl
df1.drop(to_drop, axis=1, inplace=True)

print(df1.shape)


In [ ]:
#Removing low variance columns in interest of ram
from sklearn.feature_selection import VarianceThreshold
from itertools import compress

temp = df1.drop(['D_63', 'D_64'], axis=1)

# Initialize and fit the method
vt = VarianceThreshold(threshold = float(0.1))
vt.fit(temp)

#columns with sufficient variance
keep = list(compress(temp.columns, vt.get_support()))

keep.append('D_63')
keep.append('D_64')

df1=df1[keep]
len(keep)


In [ ]:
#removing outliers
##print(df1.shape)

#df1 = df1[df1['R_6'] < df1['R_6'].quantile(0.97)]
#print(df1['R_6'].max())
#print(df1.shape)

In [ ]:
#df1.iloc[:100000,7].value_counts()
#print(df1.iloc[:,1].head())


#What type of variable for dates
#df1['S_2'] = pd.to_datetime(df1['S_2'])
#df1['S_2'] = pd.to_numeric(df1['S_2'])

#normalizing
#df1['S_2'] = (df1['S_2']-df1['S_2'].min())/(df1['S_2'].max() - df1['S_2'].min())
#print(df1['S_2'].head())

#df1['S_2'] = pd.to_timedelta(df1['S_2'])
#print(df1.iloc[:,1].dt.total_seconds())

In [ ]:
#Hot ones
df1 = pd.get_dummies(df1)

In [ ]:
#Handling missing values
#my_imputer = SimpleImputer()
#df1.iloc[:,:] = my_imputer.fit_transform(df1.iloc[:,:])

In [ ]:
X = df1.iloc[:, :].values.reshape(-1, len(df1.columns))
Y = df_label.iloc[:len(df1), 1].values.reshape(-1, 1)
print('half')
# create object for the class
log = LogisticRegression()
log.fit(X, Y) 
Y_pred = log.predict(X)

print(Y_pred, np.sum(Y_pred))
print(log.score(X, Y))




In [ ]:
cm = metrics.confusion_matrix(Y, Y_pred)

plt.figure(figsize=(9,9))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'YlGnBu');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
all_sample_title = 'Accuracy Score: {0}'.format(round(log.score(X, Y),3))
plt.title(all_sample_title, size = 15);

print('Accuracy:',round(metrics.accuracy_score(Y, Y_pred),3))
metrics.roc_curve(Y, Y_pred)
print(metrics.roc_auc_score(Y, Y_pred))

In [ ]:
#free up ram
del df1, df_label, cm
gc.collect()

In [ ]:
#run prediction on test data

print(len(keep))
#need to only load some columns due to ram limitations
df2=pd.read_parquet('/kaggle/input/amex-parquet/test_data.parquet', columns =keep)
print(df2.shape)

#hot ones
df2 = pd.get_dummies(df2)

In [ ]:
#Handling missing values via imputation
#df2.iloc[:,:] = my_imputer.fit_transform(df2.iloc[:,:])


In [ ]:
#using only most recent transaction from each customer
temp = df2.shape
df2=df2.set_index(['customer_ID'])
df2=df2.ffill()
df2=df2.bfill()
df2=df2.reset_index()

df2=df2.groupby('customer_ID').tail(1)
df2=df2.set_index(['customer_ID'])

#Drop date column since it is no longer useful
df2.drop(['S_2'],axis=1,inplace=True)

print(temp, df2.shape)

#inspecting NaN
for i in range(len(df2.columns)):
    print('Columns left with NaN:')
    if (df2.iloc[:,i].isnull().sum()/len(df2) > 0):
        print(df2.columns[i], round(df2.iloc[:,i].isnull().sum()/len(df2),2))

In [ ]:
X = df2.iloc[:, :].values.reshape(-1, len(df2.columns))
Y_pred2 = log.predict(X)
print(Y_pred2)

df2 = df2.reset_index()

final = pd.DatFrame({"customer_ID":df2.customer_ID,"prediction":Y_pred2})

print(Y_pred2, np.sum(Y_pred2))